# Inspecting FCC Political File Data (with `agate`)

In [22]:
import agate
import re

In [24]:
fcc_files = agate.Table.from_csv('fcc_opif.facilityfile.csv')

## KOMU's Political Files

Let's identify the different kinds of files KOMU has been uploading into the Political Files parent folder.

In [25]:
komu_files = fcc_files.where(lambda row: row['folder__entity'] == 'KOMU-TV')

In [26]:
len(komu_files)

748

### Agreement Forms 

First, there are agreement forms for political advertisements signed by both the facility and represenstatives of the political campaigns. The agreement could be with candidate campaign, such as [this one](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2016/State/PARSON%20FOR%20LT%20GOVERNOR%20OF%20MISSOURI/NAB_FORM.pdf) on behalf of Governor Mike Parson, or non-candidate issue campaign, such as [this one](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2014/Non-Candidate%20Issue%20Ads/PROTECT%20THE%20HARVEST-MO%20PAC/NAB_Protect_The_Harvest_PAC.pdf) on behalf of Protect the Harvest PAC. 

These forms are prescribed by the There are National Association of Broadcaster Forms. As such, it appears that KOMU's naming format for these files a couple of variations on the string `NAB`.

In [50]:
def nab_filter(row):
    return 'nab' in row['file_name'].lower() or 'n a b' in row['file_name'].lower()

In [52]:
komu_nab_forms = komu_files \
    .where(lambda row: nab_filter(row))

In [53]:
len(komu_nab_forms)

141

### Contracts

Next, there are contracts (aka, order forms) that specify the amount of paid by the advertiser for each ad, as well as the total for all the ads covered by the contract. [Here](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2016/State/PARSON%20FOR%20LT%20GOVERNOR%20OF%20MISSOURI/39109--1.pdf) is an example of a contract between KOMU and Governor Mike Parsons for $1,096.50 worth of ads that ran of July of 2016.

In [55]:
def order_form_filter(row):
    return re.match(r'(.+)?\d{5}(--\d+)?', row['file_name'])

In [56]:
komu_order_forms = komu_files \
    .where(lambda row: order_form_filter(row))

In [57]:
len(komu_order_forms)

600

### Others

There are handful of files left over.

In [58]:
len(komu_files) - (len(komu_nab_forms) + len(komu_order_forms))

7

In [59]:
other = komu_files \
    .where(lambda row: not nab_filter(row)) \
    .where(lambda row: not order_form_filter(row))

Here is what they look like.

In [71]:
other.select(['file_name', 'file_id']).print_table(max_rows=None, max_column_width=None)

| file_name                                           | file_id                              |
| --------------------------------------------------- | ------------------------------------ |
| Hillary Clinton Saturday Night Live July 23 2016    | f6d1c6c6-da97-e682-a110-da46e901086b |
| Donald Trump Saturday Night Live November 7 2015    | f22365a4-e3c6-5e71-49c3-9ba302289a11 |
| MISSOURI FARMERS CARE 7-28-8-5                      | fdafc180-5401-2488-1410-7cee726fde5e |
| Hillary Clinton Saturday Night Live October 24 2015 | 5a2e8343-76d1-e6f4-d648-2e9fd296a964 |
| Hillary Clinton Saturday Night Live October 4 2015  | 6653f862-c85f-6fe1-d52e-37c15735ca61 |
| Bernie Sanders Saturday Night Live February 6 2016  | d37a49b4-389d-7363-42f2-009330982b0e |
| Bernie Sanders Saturday Night Live June 11 2016     | 5dd1450d-7461-ff57-e505-17f49933ac9a |


The "MISSOURI FARMERS CARE 7-28-8-5" is actually a [non-candidate issue ad contract](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2014/Non-Candidate%20Issue%20Ads/Missouri%20Farmers%20Care/MISSOURI_FARMERS_CARE_7-28-8-5.pdf). For some reason they threw in some extra dashes when they named this file.

The remaining files are all notices of free time provided by facilities to political candidates, such as the time [Donald Trump appeared on Saturday Night Live](http://fcc-opif.s3.amazonaws.com/fcc_files/KOMU-TV/Political%20Files/2015/Federal/President/Donald%20Trump/Donald_Trump_Saturday_Night_Live_November_7_2015.pdf) in Nov 2015. These notices are required by [Section 73.1943 (paragraph b)](https://www.law.cornell.edu/cfr/text/47/73.1943) of Federal Regulations. 